In [ ]:
# Ich benutze hier den Kernel aus dem mamba tango enviroment.
import asyncio, time, nest_asyncio
nest_asyncio.apply()

# asyncio
-  Python-Modul für das asynchrone Programmierung
-  **kooperative** Nebenläufigkeit, keine echten parallele Aufgaben
-  Anwendungsbeispiele: gleichzeitige Abfragen von Datenbanken, Webservices, Geräten
-  Verwaltet die Aufgaben in einer Event Schleife in **einem Thread**
-  Zum starten eines asynchroner Coroutine werden **Tasks** verwendet 
- `async def` definiert eine **Coroutine** 
- `await` warten auf das Beenden einer Coroutine
- Um das Programm HauptThread nicht zu blockieren, werden Tasks benutzt


### Beispiel für synchrone Ausführung:

In [191]:
import time

def sync_task(name, duration):
    print(f"Simuliere zeitintensive Abfrage für Task {name}")
    time.sleep(duration)
    print(f"Task {name} abgeschlossen")

def main_sync():
    for i in range(5):
        sync_task(f"Task {i+1}", 1)

if __name__ == "__main__":
    start_time = time.time()
    main_sync()
    print(f"Gesamtzeit: {time.time() - start_time:.2f} Sekunden")


Simuliere zeitintensive Abfrage für Task Task 1
Task Task 1 abgeschlossen
Simuliere zeitintensive Abfrage für Task Task 2
Task Task 2 abgeschlossen
Simuliere zeitintensive Abfrage für Task Task 3
Task Task 3 abgeschlossen
Simuliere zeitintensive Abfrage für Task Task 4
Task Task 4 abgeschlossen
Simuliere zeitintensive Abfrage für Task Task 5
Task Task 5 abgeschlossen
Gesamtzeit: 5.01 Sekunden


### Beispiel für kooperative asynchrone Auführung

In [192]:
import asyncio

async def async_task(name, duration):
    print(f"Simuliere zeitintensive Abfrage für Task {name}")
    await asyncio.sleep(duration)
    print(f"Task {name} abgeschlossen")

async def main_async():
    print("Erzeuge 5 asynchrone zeitintensive Tasks:")
    tasks = [asyncio.create_task(async_task(f"Task {i+1}", 1)) for i in range(5)] 
    await asyncio.gather(*tasks) # Warte auf alle Tasks

if __name__ == "__main__":
    start_time = time.time()
    asyncio.run(main_async())
    print(f"Gesamtzeit: {time.time() - start_time:.2f} Sekunden")


Erzeuge 5 asynchrone zeitintensive Tasks:
Simuliere zeitintensive Abfrage für Task Task 1
Simuliere zeitintensive Abfrage für Task Task 2
Simuliere zeitintensive Abfrage für Task Task 3
Simuliere zeitintensive Abfrage für Task Task 4
Simuliere zeitintensive Abfrage für Task Task 5
Task Task 1 abgeschlossen
Task Task 3 abgeschlossen
Task Task 5 abgeschlossen
Task Task 2 abgeschlossen
Task Task 4 abgeschlossen
Gesamtzeit: 1.00 Sekunden


### Beispiel: Warten auf ein Event

In [194]:
async def wait_for_event(event):
    print("Warte auf ein Event. Beachte, dass Du weiterarbeiten kannst!")
    try:
        await asyncio.wait_for(event.wait(), timeout=5.0)
        print("Event was set!")
    except asyncio.TimeoutError:
        print("Timeout reached, event was not set.")

event = asyncio.Event()
async def main():
    # Start the coroutine to wait for the event
    asyncio.create_task(wait_for_event(event))   # Das ist der Grund, warum der Aufruf nichtblockierend ist!

# Run the main function
asyncio.run(main())


Warte auf ein Event. Beachte, dass Du weiterarbeiten kannst!
Event was set!


In [195]:
event.set()


## Vorteile 
- Geradeliniger Code
- einfacher zu Handhaben als Threads, weil keine echte Parallelität
- Schlanker als Threads, 3 kB vs 50 kB



## Links
https://cheat.readthedocs.io/en/latest/python/asyncio.html

https://docs.python.org/3/library/asyncio-task.html

https://www.youtube.com/watch?v=HzpdMDYEstA


## Futures
F ist ein Objekt, die zukünftige Werte oder Ausnahmen repräsentiert. F Objekts werden zur Verwaltung asynchroner Operationen genutzt. Sehe auch Vergeich mit Tasks.
F können Callback Funktionen aufrufen.


In [ ]:
async def perform_async_operation(future):
    # Simuliere eine asynchrone Operation, z.B. eine Berechnung oder Abfrage
    await asyncio.sleep(2)
    # Setze das Ergebnis des Futures nach Abschluss der Operation
    future.set_result("Operation completed successfully!")

def future_cb1(future):
    print("Ich bin die 1. CallBack Funktion!")

def future_cb2(future):
    print("Ich bin die 2. CallBack Funktion!")

future = asyncio.Future()
future.add_done_callback(future_cb1) # DONE CALLBACK
future.add_done_callback(future_cb2)

await perform_async_operation(future)
result = await future
print(result)

In [ ]:
async def set_future_value(future, value):
    # Simuliere eine asynchrone Operation
    await asyncio.sleep(1)
    future.set_result(value) #!!! Dieser Wert kann später abgefragt werden!!!

async def main():
    # Erstellen eines Future-Objekts
    future = asyncio.Future()
    
    # Starte eine Aufgabe, die den Future-Wert setzt
    asyncio.create_task(set_future_value(future, 'Hello, Future!'))

    # Warten auf das Setzen des Future-Werts
    result = await future
    print(result)

# Startet das asyncio-Event-Loop und führt die main()-Funktion aus
asyncio.run(main())


## Task (in asyncio)
Ein Task ist eine Einheit der asynchronen Ausführung, die auf einer einzigen Thread (dem Hauptthread) läuft. 
- basiert auf Coroutines
- nutzt die asyncio-Eventschleife, um nebenläufige Operationen zu managen.
- ermöglichen kooperative Nebenläufigkeit. Sie laufen nicht parallel, sondern geben die Kontrolle freiwillig zurück, wenn sie auf I/O-Operationen warten, indem sie await verwenden. Die Eventschleife entscheidet dann, welche andere Aufgabe fortgesetzt wird.
- Leichtgewicht: Tasks sind leichter als Threads, weil sie keinen eigenen Speicherraum benötigen und vom Python-Interpreter gemanagt werden.

- Verwendung: Tasks werden für I/O-gebundene Programme verwendet, wie das gleichzeitige Abwarten auf mehrere Netzwerkverbindungen, Dateioperationen oder andere asynchrone Ereignisse.
Ts planen und verwalten die Ausführung von CFs. 
- `mytask = asyncio.create_task(coro_f(params))`  
 `create_task` lässt  einer Coroutine kooperativ parallel ablaufen. TaskObjekt zu steuerung des Tasks.
- Ts werden **unmittelbar** nach erstellung ausgeführt!

In [190]:
async def say_hello():
    print("*** 2. Hello, I am the Task! ***")
    await asyncio.sleep(2)
    print("*** 5. I have finished! ***")

async def main():

    print("1. Task erstellen und ausführen") 
    task = asyncio.create_task(say_hello())
    # await asyncio.sleep(0.1)
    print("3. Doing something while waiting...") # ACHTUNG, ohne await davor, wird diese Zeile VOR Task ausgegeben. 
    await asyncio.sleep(0.9)
    print(" 4. Doing something else while waiting...")
    
    # Warten auf die Task
    await task
    print("6. Doing something after Task has finished.")


# Startet den asyncio-Event-Loop und führt main() aus
asyncio.run(main())

1. Task erstellen und ausführen
3. Doing something while waiting...
*** 2. Hello, I am the Task! ***
 4. Doing something else while waiting...
*** 5. I have finished! ***
6. Doing something after Task has finished.



### Task vs. Futures:
Task ist eine Kindklasse von Future, erbt also alles.
- Futures sind allgemeine Platzhalter für zukünftige Werte
- Task sind speziell für CF Ausführnng gedacht
- Futures müssen explizit gestartet werden mit `set_result()` oder `set_exception()`
- Task werden verwendet, eine CF in Hintegrund auszuführen
- Future wird oft verwendet, um eine Schnittstelle zu einem System zu definieren, das einen zukünftigen Wert liefert, den man manuell kontrollieren muss, wann und wie er bereitgestellt wird



### Taksgroup
`asyncio.TaskGroup` ist eine moderne (ab Python 3.11) Alternative zu `create_task()` und `asyncio.gather()`

In [ ]:
async def main():
    async with asyncio.TaskGroup() as tg:
        task1 = tg.create_task(
            say_after(2, 'hello'))

        task2 = tg.create_task(
            say_after(2.1, 'world'))

        print(f"started at {time.strftime('%X')}")

    # The await is implicit when the context manager exits.

    print(f"finished at {time.strftime('%X')}")

asyncio.run(main())

## Timeouts

In [ ]:
async def factorial(name, number):
    f = 1
    for i in range(2, number + 1):
        print(f"Task {name}: Compute factorial({number}), currently i={i}...")
        await asyncio.sleep(1)
        f *= i
    print(f"Task {name}: factorial({number}) = {f}")
    return f

async def main():
    # Schedule three calls *concurrently*:
    L = await asyncio.gather(
        factorial("A", 2),
        factorial("B", 3),
        factorial("C", 4),
    )
    print(L)

asyncio.run(main())

In [ ]:
async def main():
    try:
        async with asyncio.timeout(3):
            # Schedule three calls *concurrently*:
            L = await asyncio.gather(
                factorial("A", 2),
                factorial("B", 3),
                factorial("C", 4),
            )
        print(L)
    except TimeoutError:
        print("Einige Faktorials konnten nicht berechnet werden!")
    
asyncio.run(main())

## Warte auf mehrere Events
Weil `wait_for` nur auf ein Event warten kann, muss man `asyncio.wait` verwenden. `wait` arbeitet mit Tasks, deswegen muss man jedes `wait_for` in einem Task starten.

`wait` wirt keine Exception. Wenn ein Task nach Timeout nicht fertig ist, wird er in `pending` RW zurückgegeben. Die beendeten Tasks sind in `done`.

Wenn man als `return_when=asyncio.FIRST_COMPLETED` verwendet, und ein Event Timeout hat, wird der zugehörige Task als **BEENDET** angesehen. 


In [ ]:
import asyncio, nest_asyncio
nest_asyncio.apply()

timeout_duration = 10

async def await_event(event, name):
    print(f"warte auf {name} ")
    try:
        event = await asyncio.wait_for(event.wait(), timeout=5)
    except asyncio.TimeoutError:
      print(f"{name} timeout")


async def wait_for_2events(event1, event2):
    print("wait for events")
    t1 = asyncio.create_task(await_event(event1, "event1"), name='evt1')
    t2 = asyncio.create_task(await_event(event2, "event2"), name='evt2')

    done, pending = await asyncio.wait([t1, t2], timeout=timeout_duration, return_when=asyncio.FIRST_COMPLETED)
    for d in done: 
        print(f"{d.get_name()} was set")
    
    
    for p in pending: 
        #p.cancel()
        print(f"{p.name.get_name()} is pending")  
    

event1 = asyncio.Event()
event2 = asyncio.Event()

async def main():
    asyncio.create_task(wait_for_2events(event1, event2))   # Das ist der Grund, warum der Aufruf nichtblockierend ist!

# Run the main function
asyncio.run(main())


In [ ]:
event2.set()

In [ ]:
event1.set()

## Warten auf das Gleiche Event (z.B. new_triggerID) in mehreren Tasks.
Hat funktioniert.

In [ ]:
import asyncio

async def task_waiting_for_event(event, name, timeout):
    try:
        print(f'Task {name} wartet auf das Event mit einem Timeout von {timeout} Sekunden...')
        await asyncio.wait_for(event.wait(), timeout)
        print(f'Task {name} wurde fortgesetzt!')
    except asyncio.TimeoutError:
        print(f'Task {name} hat das Warten wegen Timeout abgebrochen.')

event = asyncio.Event()

async def main():
    # Erstelle mehrere Tasks, die auf dasselbe Event warten, mit einem Timeout
    task1 = asyncio.create_task(task_waiting_for_event(event, "A", 3))
    task2 = asyncio.create_task(task_waiting_for_event(event, "B", 5))
    task3 = asyncio.create_task(task_waiting_for_event(event, "C", 7))

asyncio.run(main())

In [ ]:
event.set()

### Perfomance Test

In [ ]:
import time 
async def task_waiting_for_event(event, name, timeout):
    try:
        event = await asyncio.wait_for(event.wait(), timeout)
        return event
        #await asyncio.sleep(1)
    except asyncio.TimeoutError:
        pass

event = asyncio.Event()

async def main():
    tasks = [asyncio.create_task(task_waiting_for_event(event, str(i), 7)) for i in range(1000)]
    start = time.time()

   # await asyncio.sleep(1)
    event.set()
    await asyncio.gather(*tasks)
    dauer = time.time() - start
    print(f"Ausführungsdauer: {dauer}")

asyncio.run(main())

In [ ]:
event.set()

# Beispiel Race Conditions mit Threads
Es ist mir noch nicht klar, wann das Problem bei Python auftritt. Man muss die Besonderheit der GIL beachten.

In [232]:
import threading
import time

# Gemeinsame Ressource
counter = 0

# Funktion, die von mehreren Threads ausgeführt wird
def increment_counter():
    global counter
    local_counter = counter
    time.sleep(0.001)  # Simuliere eine kleine Verzögerung
    local_counter += 1
    counter = local_counter

# Anzahl der Threads
num_threads = 100

# Erstelle und starte Threads
threads = []
for _ in range(num_threads):
    thread = threading.Thread(target=increment_counter)
    threads.append(thread)
    thread.start()

# Warte darauf, dass alle Threads beendet sind
for thread in threads:
    thread.join()

# Erwarteter Wert: 100, aber aufgrund von Race Conditions wird er wahrscheinlich niedriger sein
print(f"Endwert des Counters: {counter}")

Endwert des Counters: 23


## GIL Effekt
in dieser Variante tritt das Race Conditions erstaunlicher Weise nicht auf!

In [253]:
# Gemeinsame Ressource
counter = 0

# Funktion, die von mehreren Threads ausgeführt wird
def increment_counter():
    global counter
    time.sleep(0.001)  # Simuliere eine kleine Verzögerung
    counter += 1
    
# Anzahl der Threads
num_threads = 100

# Erstelle und starte Threads
threads = []
for _ in range(num_threads):
    thread = threading.Thread(target=increment_counter)
    threads.append(thread)
    thread.start()

# Warte darauf, dass alle Threads beendet sind
for thread in threads:
    thread.join()

# Erwarteter Wert: 100, aber aufgrund von Race Conditions wird er wahrscheinlich niedriger sein
print(f"Endwert des Counters: {counter}")


Endwert des Counters: 100


In [242]:
import asyncio

# Gemeinsame Ressource
counter = 0

# Coroutine, die von mehreren async Tasks ausgeführt wird
async def increment_counter():
    global counter
    local_counter = counter
    await asyncio.sleep(0.001)  # Simuliere eine kleine Verzögerung
    local_counter += 1
    counter = local_counter

async def main_async():
    global counter
    counter = 0  # Setze den Counter zurück

    # Anzahl der Tasks
    num_tasks = 100

    # Erstelle und starte Tasks
    tasks = [asyncio.create_task(increment_counter()) for _ in range(num_tasks)]

    # Warte darauf, dass alle Tasks beendet sind
    await asyncio.gather(*tasks)

    # Der Wert sollte immer 100 sein
    print(f"Endwert des Counters: {counter}")

# Starte das asyncio-Eventloop
asyncio.run(main_async())


Endwert des Counters: 1
